In [1]:
TABLE1_PATH = "../data/public/main1.csv"
TABLE2_PATH = "../data/public/main2.csv"
TABLE3_PATH = "../data/public/main3.csv"

In [2]:
# for path in (TABLE1_PATH, TABLE2_PATH, TABLE3_PATH):
#     print(f"File: {path}")
#     with open(path) as f:
#         k = 0
#         for line in f:
#             if k == 0:
#                 print(line)
#             k += 1
#         print(f"Total lines: {k}")
#     print()

In [3]:
import pandas as pd

LIMIT_SIZE = 100000
table1 = pd.read_csv(TABLE1_PATH, nrows=LIMIT_SIZE)
table2 = pd.read_csv(TABLE2_PATH, nrows=LIMIT_SIZE)
table3 = pd.read_csv(TABLE3_PATH, nrows=LIMIT_SIZE)

table1.to_csv("../data/subset_main1.csv", index=False)
table2.to_csv("../data/subset_main2.csv", index=False)
table3.to_csv("../data/subset_main3.csv", index=False)

In [4]:
print(f"Table 1 columns: {list(table1.columns)}")
print(f"Table 2 columns: {list(table2.columns)}")
print(f"Table 3 columns: {list(table3.columns)}")

Table 1 columns: ['uid', 'full_name', 'email', 'address', 'sex', 'birthdate', 'phone']
Table 2 columns: ['uid', 'first_name', 'middle_name', 'last_name', 'birthdate', 'phone', 'address']
Table 3 columns: ['uid', 'name', 'email', 'birthdate', 'sex']


In [5]:
total_columns = ["uid", "name", "birthdate", "phone", "address", "email"]

In [6]:
from app.processor import Processor

table1["name"] = table1["full_name"]
table2["name"] = table2["last_name"] + " " + table2["first_name"] + " " + table2["middle_name"]
table3["name"] = table3["name"].mapply(lambda x: x.split()[-1] + " " + x.split()[0])

formatted_table1 = Processor(table1).clean_data()
formatted_table2 = Processor(table2).clean_data()
formatted_table3 = Processor(table3).clean_data()

concat_df = pd.concat([formatted_table1[total_columns], formatted_table2[total_columns]])
concat_df = pd.concat([concat_df, formatted_table3[total_columns]])
print(len(formatted_table1), len(formatted_table2), len(formatted_table3), len(concat_df))

KeyboardInterrupt: 

In [ ]:
concat_df.sort_values(by="name").to_csv("../data/concat.csv", index=False)

In [ ]:
table1[table1["uid"] == "81c8d4d9-542c-4728-abad-e5f121f2cb7c"]

In [ ]:
table1[table1["uid"] == "0fdcf1ca-c810-4852-878a-1f769d9e7f57"]